In [1]:
import pandas as pd
from os import path, makedirs
from dotenv import load_dotenv
from unidecode import unidecode

In [2]:
load_dotenv()

True

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `03 habitação - extração.ipynb`.

In [3]:
input_dir = path.join('dados')

In [4]:
filename = path.join(input_dir, 'distritos_original.csv')
df_distritos = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype=str)
df_distritos

,id,cd_identificador_distrito,cd_identificador_subprefeitura,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal
0,distrito_municipal.8583394,8583394,5,51,MANDAQUI,MAN
1,distrito_municipal.8583395,8583395,20,52,MARSILAC,MAR
2,distrito_municipal.8583396,8583396,12,32,MOEMA,MOE
3,distrito_municipal.8583397,8583397,25,53,MOOCA,MOO
4,distrito_municipal.8583398,8583398,10,54,MORUMBI,MOR
...,...,...,...,...,...,...
91,distrito_municipal.8583485,8583485,27,47,JOSE BONIFACIO,JBO
92,distrito_municipal.8583486,8583486,28,96,LAJEADO,LAJ
93,distrito_municipal.8583487,8583487,8,48,LAPA,LAP
94,distrito_municipal.8583488,8583488,9,49,LIBERDADE,LIB


In [5]:
filename = path.join(input_dir, 'subprefeituras_original.csv')
df_subs = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype=str)
df_subs

,cd_subprefeitura,cd_identificador_subprefeitura,nm_subprefeitura
0,02,1,PIRITUBA-JARAGUA
1,03,2,FREGUESIA-BRASILANDIA
2,04,3,CASA VERDE-CACHOEIRINHA
3,05,4,SANTANA-TUCURUVI
4,06,5,JACANA-TREMEMBE
5,07,6,VILA MARIA-VILA GUILHERME
6,21,7,PENHA
7,22,8,ERMELINO MATARAZZO
8,23,9,SAO MIGUEL
9,24,10,ITAIM PAULISTA


In [6]:
filename = path.join(input_dir, 'censo_demografia_original.csv')
df_pop = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='latin1',
            dtype={'CD_DIST': str})
df_pop

,CD_DIST,NM_DIST,V01031,V01032,V01033,V01034,V01035,V01036,V01037,V01038,V01039,V01040,V01041
0,355030801,Água Rasa,3457,3966,3974,4022,4765,5504,13078,14034,11887,10700,10401
1,355030802,Alto de Pinheiros,1260,1572,1691,1745,1862,2181,4834,5671,5290,4897,6314
2,355030803,Anhanguera,4572,5178,4997,5584,6416,6256,11708,11423,10157,5740,3248
3,355030804,Aricanduva,4413,4850,4964,5693,6384,6205,12924,13701,12001,9693,8734
4,355030805,Artur Alvim,4333,5342,5321,5849,6913,6882,14024,14694,12522,10239,9447
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,355030892,Vila Medeiros,5748,6357,6422,6887,8526,8261,17593,16401,14488,12815,11436
92,355030893,Vila Prudente,4724,5116,5109,5289,6563,7950,18936,16565,13414,11348,10638
93,355030894,Vila Sônia,6314,7336,7131,7318,8595,9194,20167,19999,15282,11561,10804
94,355030895,São Domingos,4506,5084,5151,5537,6143,6385,13251,13733,11996,8986,8096


# Transformação dos dados

## Unindo as tabelas de distritos e subprefeituras

Vemos que as subprefeituras possuem duas colunas diferentes de id: `cd_identificador_subprefeitura` e `cd_subprefeitura`. Como as os valores nas duas colunas não coincidem, vamos avaliar qual delas é a mais adequada para fazermos a junção com os dados dos distritos.

In [7]:
df_subs.sort_values('cd_identificador_subprefeitura').head(5)

,cd_subprefeitura,cd_identificador_subprefeitura,nm_subprefeitura
0,02,1,PIRITUBA-JARAGUA
9,24,10,ITAIM PAULISTA
10,27,11,ITAQUERA
11,28,12,GUAIANASES
12,08,13,LAPA


In [8]:
df_distritos.sort_values('cd_identificador_subprefeitura').head(5)

,id,cd_identificador_distrito,cd_identificador_subprefeitura,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal
49,distrito_municipal.8583443,8583443,1,3,ANHANGUERA,ANH
10,distrito_municipal.8583404,8583404,1,61,PERUS,PRS
14,distrito_municipal.8583408,8583408,10,65,RAPOSO TAVARES,RTA
16,distrito_municipal.8583410,8583410,10,67,RIO PEQUENO,RPE
4,distrito_municipal.8583398,8583398,10,54,MORUMBI,MOR


Vemos que ocorrem duas coisas: a coluna `cd_identificador_subprefeitura` da tabela de distritos coincide com o valor numérico da coluna `cd_subprefeitura`, mas a coluna `cd_subprefeitura` está em formato de texto, iniciando com zeros à esquerda. Portanto, vamos adicionar à tabela de distritos uma nova coluna `cd_subprefeitura`, convertendo o valor da coluna `cd_identificador_subprefeitura` para texto e preenchendo com zeros à esquerda para que a string tenha 2 caracteres.

In [9]:
df_distritos.insert(
    3,
    'cd_subprefeitura',
    df_distritos['cd_identificador_subprefeitura'].astype(str).str.zfill(2)
)
df_distritos.head()

,id,cd_identificador_distrito,cd_identificador_subprefeitura,cd_subprefeitura,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal
0,distrito_municipal.8583394,8583394,5,05,51,MANDAQUI,MAN
1,distrito_municipal.8583395,8583395,20,20,52,MARSILAC,MAR
2,distrito_municipal.8583396,8583396,12,12,32,MOEMA,MOE
3,distrito_municipal.8583397,8583397,25,25,53,MOOCA,MOO
4,distrito_municipal.8583398,8583398,10,10,54,MORUMBI,MOR


Agora, vamos unir as tabelas de distritos e subprefeituras, utilizando a coluna `cd_subprefeitura` como chave de junção.

In [10]:
df_dist_sub = (
    df_distritos.drop(columns=['cd_identificador_subprefeitura'])
    .merge(
        df_subs.drop(columns=['cd_identificador_subprefeitura']),
        on='cd_subprefeitura',
        how='left'
    )
)

df_dist_sub.head()

,id,cd_identificador_distrito,cd_subprefeitura,cd_distrito_municipal,nm_distrito_municipal,sg_distrito_municipal,nm_subprefeitura
0,distrito_municipal.8583394,8583394,05,51,MANDAQUI,MAN,SANTANA-TUCURUVI
1,distrito_municipal.8583395,8583395,20,52,MARSILAC,MAR,PARELHEIROS
2,distrito_municipal.8583396,8583396,12,32,MOEMA,MOE,VILA MARIANA
3,distrito_municipal.8583397,8583397,25,53,MOOCA,MOO,MOOCA
4,distrito_municipal.8583398,8583398,10,54,MORUMBI,MOR,BUTANTA


Investigando o resultado no data wrangler, vemos que a junção ocorreu corretamente, com todos os distritos associados à sua respectiva subprefeitura.

## Agregando e unindo dados da população

Agora, precisamos preparar os dados da população para que possamos associá-los às subprefeituras. Vamos agrgar as colunas de população em dois grupos: população de 0 a 69 anos e população total.

In [11]:
df_pop_summary = df_pop[['CD_DIST', 'NM_DIST']].copy()
df_pop_summary

,CD_DIST,NM_DIST
0,355030801,Água Rasa
1,355030802,Alto de Pinheiros
2,355030803,Anhanguera
3,355030804,Aricanduva
4,355030805,Artur Alvim
...,...,...
91,355030892,Vila Medeiros
92,355030893,Vila Prudente
93,355030894,Vila Sônia
94,355030895,São Domingos


In [12]:
df_pop_summary.insert(
    2,
    'populacao_total',
    df_pop.iloc[:, 2:].sum(axis=1)
)

df_pop_summary

,CD_DIST,NM_DIST,populacao_total
0,355030801,Água Rasa,85788
1,355030802,Alto de Pinheiros,37317
2,355030803,Anhanguera,75279
3,355030804,Aricanduva,89562
4,355030805,Artur Alvim,95566
...,...,...,...
91,355030892,Vila Medeiros,114934
92,355030893,Vila Prudente,105652
93,355030894,Vila Sônia,123701
94,355030895,São Domingos,88868


In [13]:
df_pop_summary.insert(
    2,
    'populacao_0_69',
    df_pop_summary['populacao_total'] - df_pop['V01041']
)

df_pop_summary

,CD_DIST,NM_DIST,populacao_0_69,populacao_total
0,355030801,Água Rasa,75387,85788
1,355030802,Alto de Pinheiros,31003,37317
2,355030803,Anhanguera,72031,75279
3,355030804,Aricanduva,80828,89562
4,355030805,Artur Alvim,86119,95566
...,...,...,...,...
91,355030892,Vila Medeiros,103498,114934
92,355030893,Vila Prudente,95014,105652
93,355030894,Vila Sônia,112897,123701
94,355030895,São Domingos,80772,88868


## Unindo os dados de população por distritos

Vamos criar uma coluna com os nomes dos distritos sem acentuação e em letras maiúsculas para possibilitar a união dos dados.

In [14]:
df_pop_summary.insert(
    2,
    'nm_distrito_municipal',
    df_pop_summary['NM_DIST'].apply(lambda x: unidecode(x).upper().strip())
)

df_pop_summary

,CD_DIST,NM_DIST,nm_distrito_municipal,populacao_0_69,populacao_total
0,355030801,Água Rasa,AGUA RASA,75387,85788
1,355030802,Alto de Pinheiros,ALTO DE PINHEIROS,31003,37317
2,355030803,Anhanguera,ANHANGUERA,72031,75279
3,355030804,Aricanduva,ARICANDUVA,80828,89562
4,355030805,Artur Alvim,ARTUR ALVIM,86119,95566
...,...,...,...,...,...
91,355030892,Vila Medeiros,VILA MEDEIROS,103498,114934
92,355030893,Vila Prudente,VILA PRUDENTE,95014,105652
93,355030894,Vila Sônia,VILA SONIA,112897,123701
94,355030895,São Domingos,SAO DOMINGOS,80772,88868


Agora vamos ordenar alfabeticamente os dataframes e comparar os nomes dos distritos em cada tabela.

In [15]:
{
    dist_censo : dist_gs
    for dist_censo, dist_gs in zip(
        sorted(df_pop_summary['nm_distrito_municipal'].unique()),
        sorted(df_dist_sub['nm_distrito_municipal'].unique())
    )
}

{'AGUA RASA': 'AGUA RASA',
 'ALTO DE PINHEIROS': 'ALTO DE PINHEIROS',
 'ANHANGUERA': 'ANHANGUERA',
 'ARICANDUVA': 'ARICANDUVA',
 'ARTUR ALVIM': 'ARTUR ALVIM',
 'BARRA FUNDA': 'BARRA FUNDA',
 'BELA VISTA': 'BELA VISTA',
 'BELEM': 'BELEM',
 'BOM RETIRO': 'BOM RETIRO',
 'BRAS': 'BRAS',
 'BRASILANDIA': 'BRASILANDIA',
 'BUTANTA': 'BUTANTA',
 'CACHOEIRINHA': 'CACHOEIRINHA',
 'CAMBUCI': 'CAMBUCI',
 'CAMPO BELO': 'CAMPO BELO',
 'CAMPO GRANDE': 'CAMPO GRANDE',
 'CAMPO LIMPO': 'CAMPO LIMPO',
 'CANGAIBA': 'CANGAIBA',
 'CAPAO REDONDO': 'CAPAO REDONDO',
 'CARRAO': 'CARRAO',
 'CASA VERDE': 'CASA VERDE',
 'CIDADE ADEMAR': 'CIDADE ADEMAR',
 'CIDADE DUTRA': 'CIDADE DUTRA',
 'CIDADE LIDER': 'CIDADE LIDER',
 'CIDADE TIRADENTES': 'CIDADE TIRADENTES',
 'CONSOLACAO': 'CONSOLACAO',
 'CURSINO': 'CURSINO',
 'ERMELINO MATARAZZO': 'ERMELINO MATARAZZO',
 'FREGUESIA DO O': 'FREGUESIA DO O',
 'GRAJAU': 'GRAJAU',
 'GUAIANASES': 'GUAIANASES',
 'IGUATEMI': 'IGUATEMI',
 'IPIRANGA': 'IPIRANGA',
 'ITAIM BIBI': 'ITAIM BIB

Todos os valores parecem corretos. Vamos ter certeza utilizando uma list comprehension para comparar os nomes dos distritos em cada tabela.

In [16]:
any([dist_censo != dist_gs
    for dist_censo, dist_gs in zip(
        sorted(df_pop_summary['nm_distrito_municipal'].unique()),
        sorted(df_dist_sub['nm_distrito_municipal'].unique())
    )])

False

Todos os valores coincidem, então podemos fazer a junção dos dataframes utilizando a coluna `nm_distrito` como chave de junção.

In [17]:
df_pop_summary = (
    df_pop_summary
    .merge(
        df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
        left_on='nm_distrito_municipal',
        right_on='nm_distrito_municipal',
        how='left'
    )
)

df_pop_summary

,CD_DIST,NM_DIST,nm_distrito_municipal,populacao_0_69,populacao_total,cd_subprefeitura,nm_subprefeitura
0,355030801,Água Rasa,AGUA RASA,75387,85788,25,MOOCA
1,355030802,Alto de Pinheiros,ALTO DE PINHEIROS,31003,37317,11,PINHEIROS
2,355030803,Anhanguera,ANHANGUERA,72031,75279,01,PERUS
3,355030804,Aricanduva,ARICANDUVA,80828,89562,26,ARICANDUVA-FORMOSA-CARRAO
4,355030805,Artur Alvim,ARTUR ALVIM,86119,95566,21,PENHA
...,...,...,...,...,...,...,...
91,355030892,Vila Medeiros,VILA MEDEIROS,103498,114934,07,VILA MARIA-VILA GUILHERME
92,355030893,Vila Prudente,VILA PRUDENTE,95014,105652,29,VILA PRUDENTE
93,355030894,Vila Sônia,VILA SONIA,112897,123701,10,BUTANTA
94,355030895,São Domingos,SAO DOMINGOS,80772,88868,02,PIRITUBA-JARAGUA
